# OpenStreetMap Data Case Study (St. Louis)

## Map Area

St. Louis, MO, United States
- [Data source](https://mapzen.com/data/metro-extracts/metro/saint-louis_missouri/)

This map was chosen since I live in this area.

First, all required libraried are impoted.

In [1]:
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re
import pandas as pd
import sqlite3 as sql
import os

## Problems encountered with the data

After downloading the data, I performed preliminary audition of the data using the included functions (count_tags, key_type, process_key_values). After the preliminary audition, four major problems were detected:
- Abbreviated street names such as : 'S Cedar Bluff Dr', 'Ogle Rd'
- NHD type of data in tags such as: 'NHD:FCode'
- tiger type of data in tags such as: 'tiger:name_type_1'
- gnis type of data in tags such as : 'gnis:ST_num'
In the foloowing sections I will explain what I have done to solve these problems.

In [2]:
def count_tags(filename):
    tags_counted = {}
    context = ET.iterparse(filename, events=("start", ))
    for event, elem in context:
        if elem.tag not in tags_counted:
            tags_counted[elem.tag] = 1
        else:
            tags_counted[elem.tag] = tags_counted[elem.tag]+1
    return tags_counted


In [3]:
#data_file = 'stlouis-sample.osm'
data_file = 'saint-louis_missouri.osm'

count_tags(data_file)

{'bounds': 1,
 'member': 13288,
 'nd': 1965462,
 'node': 1729283,
 'osm': 1,
 'relation': 1034,
 'tag': 1055373,
 'way': 167289}

In [5]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if lower.search(k):
            keys['lower'] = keys['lower']+1
        elif lower_colon.search(k):
            keys['lower_colon'] = keys['lower_colon']+1
        elif problemchars.search(k):
            keys['problemchars'] = keys['problemchars']+1
        elif k.startswith("NHD"):
            keys['NHD'] = keys['NHD']+1
            #print (k)
            #print (element.attrib['v'])
            #print ('++++++++++++')
        else:
            keys['other'] = keys['other']+1
            #print (k)
            #print (element.attrib['v'])
            #print ('++++++++++++')
    return keys



def process_k_values(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0,"NHD": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_k_values(data_file)
pprint.pprint(keys)

{'NHD': 45522,
 'lower': 415885,
 'lower_colon': 567701,
 'other': 26264,
 'problemchars': 1}


In [6]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", 'Circle', 'Way', 'Plaza']


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def is_street_name(elem):
    return (elem.attrib['k']=="addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "rb")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types




In [7]:
audit(data_file)

defaultdict(set,
            {'100': {'Highway 100'},
             '109': {'MO 109'},
             '111': {'State Route 111'},
             '143': {'12495 State Route 143'},
             '157': {'South State Route 157'},
             '2085': {'West County Center #2085'},
             '21': {'Business 21'},
             '296-7050': {'1253 Water Tower PlaceArnold, MO 63010(636) 296-7050'},
             '300': {'Olive Blvd #300'},
             '416-0707': {'3501 Lemay Ferry RoadSaint Louis, MO 63125-4425(314) 416-0707'},
             '464-3128': {'6116 2nd StKimmswick, Missouri 63052(636) 464-3128'},
             '50': {'E. US Highway 50',
              'W. US Highway 50',
              'West US Highway 50'},
             '544-0411': {'1300 Lemay Ferry RoadSaint Louis, MO 63125-2403(314) 544-0411'},
             '631-0310': {'1310 Lemay FerryLemay, MO 63125(314) 631-0310'},
             '631-2866': {'260 Hoffmeister Ave, Saint Louis, MO(314) 631-2866'},
             '631-3333': {'1101 Lem

## Abrbreviated street name
A list of expected street names were provided to a function (update_name) and based on a dictionary of abbreviations, the function changes abbreviated street names to full expected ones. For example:
- 'S Cedar Bluff Dr' was changed to 'S Cedar Bluff Drive'
- 'Ogle Rd' was changed to 'Ogle Road'

In [2]:
mapping = { "St": "Street", "St.": "Street", "Ave": "Avenue","AVE": "Avenue", "Ave.": "Avenue",
           "Av": "Avenue", "Av.": "Avenue","Rd.": "Road","Rd": "Road", "RD": "Road", "RD.": "Road", 
           "Dr.": "Drive", "Dr": "Drive", "Ctr": "Centre", "Ctr.": "Centre","Cir": "Circle", 
           "Cir.": "Circle", "Ct": "Court", "Ct.": "Court", "Cts": "Court","Cts.": "Court", 
           "PKWY":"Parkway", "Pkwy":"Parkway", "Pky":"Parkway", 'Blvd':"Boulevard", 'Blvd.':"Boulevard" ,
           'Ln':"Lane" }
def update_name(name):
    name = name.strip()
    for key in mapping:
        if name.endswith(key):
            name = re.sub(key, mapping[key], name)
    return name

## NHD
Based on the information found on the open street map [wikipedia](http://wiki.openstreetmap.org/wiki/Main_Page) page, NHD stands for National Hydrography Dataset. Howere the specific page for [NHD](http://wiki.openstreetmap.org/wiki/National_Hydrography_Dataset) was tagged for clean up at the time of this study. Therefore, The data for the NHD tags was not changed to avoid introducing new errors. However, the FCODEs were manually check to ensure they correspond to the listed feature types and no error was spotted. 

## TIGER and GNIST
the wikipedia page gor [GNIST](http://wiki.openstreetmap.org/wiki/USGS_GNIS) states that GNIST stands for Geographic Names Information System. The GNIST data is rich and contains many attributes. However, because of the discrepancy in importing data at different points in time, some data wrangling is required to make the data uniform. For example both key of 'state_id' and 'ST_num' refer to 'State FIPS code'. A function called gnis_tiger_corrector corrects these issues.
TIGER referes to the Topologically Integrated Geographic Encoding and Referencing system according to its wiki [page](http://wiki.openstreetmap.org/wiki/TIGER). The TIGER data in our dataset introduces several repetitions. For example, name_1=
Elder Court Drive and tiger:name_base_1=Elder Court and tiger:name_type_1 = Dr, which introduces several rows refering to the name. THerefore the gnis_tiger_corrector function remove the rows that talk about name_base and name_type.

In [3]:
# http://wiki.openstreetmap.org/wiki/USGS_GNIS
gnis_dict ={'Class':'Feature Class name',
            'County' : 'County name',
            'County_num' : 'County FIPS code',
            'ST_alpha' : 'State name (2-Letter)',
            'ST_num' : 'State FIPS code',
            'county_id' : 'County FIPS code',
            'state_id' : 'State FIPS code'}
def correct_key(key):
    if key in gnis_dict:
        key = gnis_dict[key]
    return key

def gnis_tiger_corrector(tags_df):
    gnis_subset = tags_df[tags_df.type=='gnis']
    gnis_subset['key']= gnis_subset['key'].apply(correct_key)
    tags_df[tags_df.type=='gnis'] = gnis_subset
    tags_df = tags_df[tags_df.key!='name_base_1']
    tags_df = tags_df[tags_df.key!='name_base_2']
    tags_df = tags_df[tags_df.key!='name_type_1']
    tags_df = tags_df[tags_df.key!='name_type_2']
    return tags_df

## Making csv files
In this section, data for nodes(locations) and ways(road) are imported and stored into panda dataframes to be written as csv files to be imported into the sql data base.

In [10]:
# Column names for each csv file:
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [17]:
'''
The "node" data frame holds a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored
'''
def nodes_maker(osmfile):
    osm_file = open(osmfile, "rb")
    nodes_dict = defaultdict(list)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node":
            for field in NODE_FIELDS:
                try:
                    nodes_dict[field].append(elem.attrib[field])
                except:
                    print(" Returned unknown for attribute: ",field)
                    nodes_dict[field].append(float('nan'))
    return nodes_dict
    

In [18]:
'''
The "node_tags" dataframe holds rows, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.
'''
def nodes_tags_maker(osmfile):
    osm_file = open(osmfile, "rb")
    nodes_tags_dict = defaultdict(list)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node":
            for subtag in elem.iter("tag"):
                if not problemchars.search(subtag.attrib['k']):
                    nodes_tags_dict['id'].append(elem.attrib['id'])
                    k= subtag.attrib['k']
                    colon = lower_colon.search(k)
                    if colon:
                        k_split = k.split(":",1)
                        nodes_tags_dict['key'].append(k_split[1])
                        nodes_tags_dict['type'].append(k_split[0])
                    else:
                        nodes_tags_dict['key'].append(k)
                        nodes_tags_dict['type'].append('regular')
                    nodes_tags_dict['value'].append(subtag.attrib['v'])
    return nodes_tags_dict
   

In [19]:
'''
The "way" dataframe holds a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored
'''
def ways_maker(osmfile):
    osm_file = open(osmfile, "rb")
    ways_dict = defaultdict(list)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            for field in WAY_FIELDS:
                try:
                    ways_dict[field].append(elem.attrib[field])
                except:
                    #print(" Returned unknown for attribute: ",atb)
                    ways_dict[field].append(float('nan'))
    return ways_dict
    

In [20]:
'''
The "way_tags" dataframe again holds data, following the exact same rules as
for "node_tags".
'''
def ways_tags_maker(osmfile):
    osm_file = open(osmfile, "rb")
    ways_tags_dict = defaultdict(list)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            for subtag in elem.iter("tag"):
                if not problemchars.search(subtag.attrib['k']):
                    ways_tags_dict['id'].append(elem.attrib['id'])
                    k= subtag.attrib['k']
                    colon = lower_colon.search(k)
                    if colon:
                        k_split = k.split(":",1)
                        ways_tags_dict['key'].append(k_split[1])
                        ways_tags_dict['type'].append(k_split[0])
                    else:
                        ways_tags_dict['key'].append(k)
                        ways_tags_dict['type'].append('regular')
                    ways_tags_dict['value'].append(subtag.attrib['v'])
    return ways_tags_dict
   

In [21]:
'''
dditionally, the "way_nodes". "way_nodes" dataframe holds rows,
one for each nd child tag.  Each row should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element
'''
def ways_nodes_maker(osmfile):
    osm_file = open(osmfile, "rb")
    ways_nodes_dict = defaultdict(list)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            pos = 0
            for subtag in elem.iter("nd"):
                if not problemchars.search(subtag.attrib['ref']):
                    ways_nodes_dict['id'].append(elem.attrib['id'])
                    ways_nodes_dict['node_id'].append(subtag.attrib['ref'])
                    ways_nodes_dict['position'].append(pos)
                    pos = pos+1

    return ways_nodes_dict
   

First, dictionaries are made with the required columns as keys.

In [22]:
nodes_dict= nodes_maker(data_file)
nodes_tags_dict= nodes_tags_maker(data_file)
ways_dict = ways_maker(data_file)
ways_tags_dict = ways_tags_maker(data_file)
ways_nodes_dict = ways_nodes_maker(data_file)

 Returned unknown for attribute:  user
 Returned unknown for attribute:  uid
 Returned unknown for attribute:  user
 Returned unknown for attribute:  uid


Then, the dictionaries are transformed into dataframes

In [33]:
nodes_tags = pd.DataFrame(nodes_tags_dict)
nodes = pd.DataFrame(nodes_dict)
ways = pd.DataFrame(ways_dict)
ways_tags = pd.DataFrame(ways_tags_dict)
ways_nodes = pd.DataFrame(ways_nodes_dict)
nodes = nodes.fillna(method='ffill')
ways = ways.fillna(method='ffill')

At this stage, street names are corrected using the update_name function to correct for abbreviations. Additionall,  the gnis_tiger_corrctor function is applied to correct the GNIS and TIGER rows as explained previously.

In [34]:
nodes_tags['value'] = nodes_tags['value'].apply(update_name)
ways_tags['value'] = ways_tags['value'].apply(update_name)
ways_tags = gnis_tiger_corrector(ways_tags)
nodes_tags = gnis_tiger_corrector(nodes_tags)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Parts of the dataframes are inspected and checked with raw data for quality control.

In [35]:
nodes.head(10)

,changeset,id,lat,lon,timestamp,uid,user,version
0,37181195,29409586,38.6239891,-90.2035946,2016-02-13T06:36:43Z,3519193,banjavjs,40
1,37181195,29409587,38.6242387,-90.204571,2016-02-13T06:36:43Z,3519193,banjavjs,46
2,37181195,29409588,38.6247124,-90.2069082,2016-02-13T06:36:43Z,3519193,banjavjs,41
3,37181195,29409589,38.6251268,-90.2084583,2016-02-13T06:36:43Z,3519193,banjavjs,41
4,37181195,29409590,38.6263375,-90.2121813,2016-02-13T06:36:43Z,3519193,banjavjs,43
5,255409,29409591,38.6270773,-90.2157044,2008-12-08T08:16:19Z,9065,brianboru,40
6,37181195,29409593,38.6277329,-90.2182167,2016-02-13T06:36:43Z,3519193,banjavjs,41
7,37181195,29409633,38.6321968,-90.2914039,2016-02-13T06:36:43Z,3519193,banjavjs,42
8,12913007,29409634,38.6322875,-90.292412,2012-08-30T04:05:54Z,10786,stucki1,40
9,37181195,29409635,38.6324014,-90.2934867,2016-02-13T06:36:43Z,3519193,banjavjs,40


In [36]:
nodes_tags.head()

,id,key,type,value
0,33053227,ref,regular,243C
1,33053227,highway,regular,motorway_junction
2,33053242,ref,regular,241A
3,33053242,highway,regular,motorway_junction
4,33056507,highway,regular,traffic_signals


In [37]:
ways.head()

,changeset,id,timestamp,uid,user,version
0,21431926,4630360,2014-04-01T04:02:47Z,262151,ToeBee,34
1,22015454,4984188,2014-04-29T05:21:02Z,262151,ToeBee,19
2,36762884,4984291,2016-01-23T18:04:57Z,3519193,banjavjs,13
3,40192973,4984322,2016-06-21T22:25:31Z,83518,Millbrooky,10
4,26105104,4984405,2014-10-15T21:01:38Z,83518,Millbrooky,31


In [38]:
ways_tags.head()

,id,key,type,value
0,4630360,hgv,regular,designated
1,4630360,ref,regular,I 64;US 40
2,4630360,lanes,regular,3
3,4630360,oneway,regular,yes
4,4630360,highway,regular,motorway


In [39]:
ways_nodes.head(10)

,id,node_id,position
0,4630360,313035047,0
1,4630360,1889319980,1
2,4630360,1889321000,2
3,4630360,313034719,3
4,4630360,560637753,4
5,4630360,560637759,5
6,4630360,1889320998,6
7,4630360,29410038,7
8,4630360,560637765,8
9,4630360,29410039,9


## Format change
To store the data as sql tables, data must be in the required format.

Order of the columns is also important and should match the order of the sql tables specified by the schema.

In [40]:
nodes.dtypes

changeset    object
id           object
lat          object
lon          object
timestamp    object
uid          object
user         object
version      object
dtype: object

In [41]:
nodes_int_columns = ['id', 'uid', 'version', 'changeset']
nodes_float_columns= ['lat', 'lon']
nodes_tags_int_columns = ['id']
ways_int_columns = ['id', 'uid', 'changeset']
ways_tags_int_columns = ['id']
ways_nodes_int_columns = ['id','node_id', 'position']

def format_corrector(dataframe, columns, format_func):
    for col in columns:
        dataframe[col] = dataframe[col].apply(format_func)
    return dataframe

In [42]:
nodes = format_corrector(nodes, nodes_int_columns, int)
nodes = format_corrector(nodes, nodes_float_columns, float)
nodes_tags = format_corrector(nodes_tags, nodes_tags_int_columns, int)
ways = format_corrector(ways, ways_int_columns, int)
ways_tags = format_corrector(ways_tags, ways_tags_int_columns, int)
ways_nodes = format_corrector(ways_nodes, ways_nodes_int_columns, int)
nodes = nodes[['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']]
nodes_tags = nodes_tags[['id', 'key', 'value', 'type']]
ways = ways[['id', 'user', 'uid', 'version', 'changeset', 'timestamp']]
ways_tags = ways_tags[['id', 'key', 'value', 'type']]
ways_nodes = ways_nodes[['id', 'node_id', 'position']]

In [43]:
nodes.dtypes

id             int64
lat          float64
lon          float64
user          object
uid            int64
version        int64
changeset      int64
timestamp     object
dtype: object

## Save  pandas dataframes as csv

In [44]:
nodes.to_csv('nodes.csv', index=False, header=False)
nodes_tags.to_csv('node_tags.csv', index=False, header=False)
ways.to_csv('ways.csv', index=False, header=False)
ways_tags.to_csv('ways_tags.csv', index=False, header=False)
ways_nodes.to_csv('ways_nodes.csv', index=False, header=False)

## Data overview and analysis

### File sizes

- The  main OSM XML file is 360.2 Mb
- The nodes.csv file is 135.9 Mb
- The nodes_tags.csv file is 2.9 Mb
- The ways.csv file is 9.4 Mb
- The ways_tags.csv file is 31.7 Mb
- The ways_nodes.csv file is 42.1 Mb
- The SQL databse is 201.5 MB

In [56]:
round(os.path.getsize('stl-map.db')/(1024*1024.0),1)


201.5

### Number of nodes

Based on the sql query there are 1729283 nodes in the data base.

In [63]:
db = sql.connect("stl-map.db")
#db.row_factory = lambda cursor, row: row[0]
c = db.cursor()
QUERY = "SELECT count(*) FROM nodes;"
result = c.execute(QUERY).fetchall()
print(result)

[(1729283,)]


### Number of ways

Based on the sql query there are 167289 ways in the data base.

In [64]:
c = db.cursor()
QUERY = "SELECT count(*) FROM ways;"
result = c.execute(QUERY).fetchall()
print(result)

[(167289,)]


### Number of unique users
Based on the sql query 1074 unique user contributed to the nodes data base and 873 unique user contributed to the nodes data base.

In [85]:
c = db.cursor()
QUERY = "SELECT  count(*) FROM (SELECT DISTINCT uid FROM nodes) as unique_users ;"
result = c.execute(QUERY).fetchall()
print(result)

[(1074,)]


In [86]:
c = db.cursor()
QUERY = "SELECT  count(*) FROM (SELECT DISTINCT uid FROM ways) as unique_users ;"
result = c.execute(QUERY).fetchall()
print(result)

[(873,)]


### Number of fueling nodes
Based on the sql query there are 132 fueling nodes in the St.Louis area.

In [91]:
c = db.cursor()
QUERY = "SELECT  count(*) FROM nodes_tags WHERE nodes_tags.value='fuel';"
result = c.execute(QUERY).fetchall()
print(result)

[(132,)]


### Top 5 amenities
 The top amenity in the St.Louis area is pace of worship with 1529 nodes, rank 2 to 5 are schools, grave yards, parkings, and restaurants.

In [96]:
c = db.cursor()
QUERY = "SELECT value, count(*) as num FROM nodes_tags WHERE key='amenity' GROUP BY value ORDER BY num desc LIMIT 5;"
result = c.execute(QUERY).fetchall()
print(result)

[('place_of_worship', 1529), ('school', 911), ('grave_yard', 528), ('parking', 431), ('restaurant', 411)]


### Top 5 cuisins near downtown st. Louis
Base on latitude and longitude of the downtown St.Louis area, top cuisines are American, Italian, pizza, barbecue, and burger restaurants. There are 6 American restaurants in the neighborhood.

In [103]:
c = db.cursor()
QUERY = "SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags , nodes, \
    (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i \
    WHERE nodes_tags.id=i.id and nodes_tags.id=nodes.id and nodes_tags.key='cuisine' \
    and nodes.lat>38.6171589 and nodes.lat<38.6349988 and nodes.lon>-90.2091266 and nodes.lon<-90.181812 \
GROUP BY nodes_tags.value ORDER BY num DESC limit 5;"
result = c.execute(QUERY).fetchall()
print(result)

[('american', 6), ('italian', 6), ('pizza', 3), ('barbecue', 2), ('burger', 2)]


## Other ideas

Based on my observation, the number of contributors can be dramatically increase by implementing some programs using schools. One can provide guideline on how students should import data into the openstreetmap and what standards need to be kept. If several highschools in each area assign data addition as tasks in their progrramming classes, a lot of data can be collected and imported.

Benefits:
- Free and precise workforce to add the data
- A great experience for the students to get involved in a real life data analysis.

Anticipated problems:
- Needs some man power to establish the program and advertise it to schools.
- Quality of the data might vary due to the variation in interest of students.
